# Random Prediction

Dieses Modell nimmt eine Liste von Parts und erstellt daraus zufällig einen Graphen. Da die Anzahl der Möglichkeiten aufgrund der Zyklenfreiheit sehr begrenzt ist, soll dieses Modell als Baseline dienen.

## Setup

In [1]:
import os
from collections import Counter
import torch
import time
import random
from matplotlib import pyplot as plt
import numpy as np
from torch import nn
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from abc import ABC, abstractmethod
from typing import Set, List, Tuple
from graph_loader import load_graphs
from graph import Graph
from part import Part
from node import Node
from typing import Dict, List, Set, Tuple, Union


In [2]:
# MPS for Metal acceleration for Mac
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.has_mps else "cpu") 
SEED = 0
random.seed(SEED)
# setgrad = lambda g, *ms: [setattr(p,'requires_grad', g) for m in ms for p in m.parameters() ]  

print(device)

mps


/var/folders/42/x47cnz7x2v98xvk5lhdybbnm0000gn/T/ipykernel_61522/3632339911.py:2: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.has_mps else "cpu")


## Custom Dataset
Wir erstellen ein custom Dataset.

In [3]:
class GraphDataset(Dataset):
    def __init__(self, file_path: str, train=False, validation=False, test=False, seed=42):
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Dataset file not found at {file_path}")

        self.graphs = load_graphs(file_path)

        if sum([train, validation, test]) != 1:
            raise ValueError("Exactly one of 'train', 'validation', or 'test' must be True.")

        # Split: 70% training, 15% validation, 15% test
        train_graphs, test_graphs = train_test_split(self.graphs, test_size=0.3, random_state=seed)
        validation_graphs, test_graphs = train_test_split(test_graphs, test_size=0.5, random_state=seed)

        if train:
            self.graphs = train_graphs
        elif validation:
            self.graphs = validation_graphs
        elif test:
            self.graphs = test_graphs

        
    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        # return parts und graphen
        return self.graphs[idx].get_parts(), self.graphs[idx]



In [4]:
training_set = GraphDataset("data/graphs.dat", train = True, seed=SEED)
validation_set = GraphDataset("data/graphs.dat", validation = True, seed=SEED)
testing_set = GraphDataset("data/graphs.dat", test = True, seed=SEED)

Überprüfen, ob die Anzahl der Parts immer größer als 1 ist --> JA

In [5]:
for parts, graph in testing_set:
    number_of_parts = len(parts)
    if number_of_parts == 0:
        print("No parts found")
        break
    elif number_of_parts == 1:
        print("1 part found")
        break
print("Number of parts always greater than 1")

Number of parts always greater than 1


## Model: Random prediction

In [6]:
from evaluation import MyPredictionModel
import random
from graph import Graph 

class RandomGraphPredictionModel(MyPredictionModel):
    def predict_graph(self, parts: Set[Part]) -> Graph:
        """
        Predicts a random graph from the given set of parts.
        :param parts: Set of Part objects.
        :return: A randomly generated Graph object.
        """

        # create empty graph
        graph = Graph()

        # shuffle parts
        parts_random = list(parts)
        random.shuffle(parts_random)

        if len(parts_random) == 0:
            return graph
        elif len(parts_random) == 1:
            # TODO: implement case of only one part/node without edges
            return graph
        else: 
            # more than 1 part. Initialize graph
            part_1 = parts_random[0]
            part_2 = parts_random[1]
            graph.add_undirected_edge(part_1, part_2)
            # remove first two parts from list
            parts_random = parts_random[2:]
        # more than 1 part --> add edges

        # add edges to graph
        for part in parts_random:
            # get random existing part from graph and connect it to the current part
            random_part = random.choice(list(graph.get_parts()))
            graph.add_undirected_edge(part, random_part)

        return graph

## Evaluation

In [7]:
testing_list = []
for parts, graph in testing_set: 
    tuple = (parts, graph)
    testing_list.append(tuple)

len(testing_list)

1674

In [8]:
from evaluation import evaluate

random_graph_builder = RandomGraphPredictionModel()
accuracy = evaluate(random_graph_builder, testing_list)
print("Accuracy of RandomGraphPredictionModel: ", accuracy)


Accuracy of RandomGraphPredictionModel:  70.71368108713263
